# 在 单-三重态量子比特系统中，采用不同脉冲数实现单量子比特任意门的计算结果

In [29]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

error_min_list = [] # max of infid

for i in range(len(u_mats)):
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(eval_x, eval_y[i])]))
    error_min_list.append(error)
print('mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

mean of error is： 0.6272645430597421

error_min_list:
 [0.26501825674997403, 0.9703480064140677, 0.46362872698125035, 0.9921292061365249, 0.14847172312148949, 0.8687930253120443, 0.27343269850037943, 0.8281934417617, 0.3429878258201977, 0.8462970834383421, 0.566951668143233, 0.8291968436540613, 0.469273761710431, 0.12838191137661747, 0.00571535245808108, 0.8672364525102665, 0.8927478380606465, 0.9028077610950185, 0.7837305248272066, 0.956468285694717, 0.11821511035252008, 0.0689915347784884, 0.952254833218876, 0.994922688607323, 0.4121172741003494, 0.9951391731723069, 0.9072755141340463, 0.8099442686456212, 0.3940456599880442, 0.8863070271255237, 0.8501585939366474, 0.281283306085752]


In [10]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
# circ += gate_0('01').on(0)
# circ += gate_0('02').on(0)
# circ += gate_0('03').on(0)
# circ += gate_0('04').on(0)
# circ += gate_0('05').on(0)
# circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 0.24582932250260003 iter_num: 3999
进度: 2/32  error_min: 0.6703344727374386 iter_num: 3999
进度: 3/32  error_min: 0.4496484743380629 iter_num: 3999
进度: 4/32  error_min: 0.6598928165176515 iter_num: 3999
进度: 5/32  error_min: 0.09700271739685162 iter_num: 3999
进度: 6/32  error_min: 0.5722075611926492 iter_num: 3999
进度: 7/32  error_min: 0.24415761576134842 iter_num: 3999
进度: 8/32  error_min: 0.23960486978034334 iter_num: 3999
进度: 9/32  error_min: 0.32138613797941495 iter_num: 3999
进度: 10/32  error_min: 0.407950071870291 iter_num: 3999
进度: 11/32  error_min: 0.5621627145242204 iter_num: 3999
进度: 12/32  error_min: 0.6943971131031069 iter_num: 3999
进度: 13/32  error_min: 0.6494657765758315 iter_num: 3999
进度: 14/32  error_min: 0.023259309046758325 iter_num: 3999
进度: 15/32  error_min: 0.004080187458626483 iter_num: 3999
进度: 16/32  error_min: 0.5236322666148397 iter_num: 3999
进度: 17/32  error_min: 0.36685038196109476 iter_num: 3999
进度: 18/32  error_min: 0.008555692058092257 iter_

In [11]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
# circ += gate_0('02').on(0)
# circ += gate_0('03').on(0)
# circ += gate_0('04').on(0)
# circ += gate_0('05').on(0)
# circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 0.18778399892822117 iter_num: 3999
进度: 2/32  error_min: 0.15605287245380872 iter_num: 3999
进度: 3/32  error_min: 0.2736579900172741 iter_num: 3999
进度: 4/32  error_min: 0.5503106382610887 iter_num: 3999
进度: 5/32  error_min: 0.0469588441893376 iter_num: 3999
进度: 6/32  error_min: 0.14476512760846094 iter_num: 3999
进度: 7/32  error_min: 0.13597629865733019 iter_num: 3999
进度: 8/32  error_min: 0.037006042888029 iter_num: 3999
进度: 9/32  error_min: 0.3139060701213682 iter_num: 3999
进度: 10/32  error_min: 0.20281119248693813 iter_num: 3999
进度: 11/32  error_min: 0.41726655417762215 iter_num: 3999
进度: 12/32  error_min: 0.3933947248581575 iter_num: 3999
进度: 13/32  error_min: 0.10397192014797352 iter_num: 3999
进度: 14/32  error_min: 0.044827821438157 iter_num: 3999
进度: 15/32  error_min: 1.4465259981610856e-05 iter_num: 3999
进度: 16/32  error_min: 0.027015541403967336 iter_num: 3999
进度: 17/32  error_min: 0.23023604776614837 iter_num: 3999
进度: 18/32  error_min: 0.055329611409609125 it

In [12]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
# circ += gate_0('03').on(0)
# circ += gate_0('04').on(0)
# circ += gate_0('05').on(0)
# circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 0.2687362796012267 iter_num: 3999
进度: 2/32  error_min: 0.010398661602562642 iter_num: 3999
进度: 3/32  error_min: 0.23972004295449012 iter_num: 3999
进度: 4/32  error_min: 0.013601649184351472 iter_num: 3999
进度: 5/32  error_min: 0.09727191125357382 iter_num: 3999
进度: 6/32  error_min: 0.09658664757452451 iter_num: 3999
进度: 7/32  error_min: 0.39156221217934206 iter_num: 3999
进度: 8/32  error_min: 0.03116913861756543 iter_num: 3999
进度: 9/32  error_min: 0.23261370870503162 iter_num: 3999
进度: 10/32  error_min: 9.347177719054578e-06 iter_num: 3580
进度: 11/32  error_min: 0.28320665933444433 iter_num: 3999
进度: 12/32  error_min: 0.10417744059862333 iter_num: 3999
进度: 13/32  error_min: 9.723921325033302e-06 iter_num: 2273
进度: 14/32  error_min: 0.0008854730320756454 iter_num: 3999
进度: 15/32  error_min: 0.005649610096432722 iter_num: 3999
进度: 16/32  error_min: 0.01621255382737974 iter_num: 3999
进度: 17/32  error_min: 0.13385245764688214 iter_num: 3999
进度: 18/32  error_min: 9.96753142

In [13]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
# circ += gate_0('04').on(0)
# circ += gate_0('05').on(0)
# circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 0.23436507850590937 iter_num: 3999
进度: 2/32  error_min: 0.051828259285021105 iter_num: 3999
进度: 3/32  error_min: 0.05116006086161329 iter_num: 3999
进度: 4/32  error_min: 0.03195895606103483 iter_num: 3999
进度: 5/32  error_min: 9.801835487932564e-06 iter_num: 2646
进度: 6/32  error_min: 0.010659088344121326 iter_num: 3999
进度: 7/32  error_min: 0.10274482981702593 iter_num: 3999
进度: 8/32  error_min: 6.915588595224342e-06 iter_num: 1053
进度: 9/32  error_min: 0.10127830937341564 iter_num: 3999
进度: 10/32  error_min: 7.1330841521222155e-06 iter_num: 585
进度: 11/32  error_min: 0.2637590922875386 iter_num: 3999
进度: 12/32  error_min: 0.041564703664677016 iter_num: 3999
进度: 13/32  error_min: 9.247968462355871e-06 iter_num: 1973
进度: 14/32  error_min: 7.093253096779328e-06 iter_num: 1308
进度: 15/32  error_min: 9.27304117903649e-06 iter_num: 455
进度: 16/32  error_min: 0.006051383503706709 iter_num: 3999
进度: 17/32  error_min: 8.949383219269968e-06 iter_num: 1786
进度: 18/32  error_min: 3.8

In [31]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
# circ += gate_0('05').on(0)
# circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 8.764936255789557e-06 iter_num: 349
进度: 2/32  error_min: 8.68173799262184e-06 iter_num: 3488
进度: 3/32  error_min: 9.705969495565903e-06 iter_num: 600
进度: 4/32  error_min: 0.030599604828755678 iter_num: 3999
进度: 5/32  error_min: 2.2866162391110834e-05 iter_num: 3999
进度: 6/32  error_min: 5.907934756299582e-06 iter_num: 970
进度: 7/32  error_min: 0.08328405180081233 iter_num: 3999
进度: 8/32  error_min: 9.293576025859451e-06 iter_num: 1753
进度: 9/32  error_min: 0.00018011514149873786 iter_num: 3999
进度: 10/32  error_min: 3.654639651795577e-06 iter_num: 355
进度: 11/32  error_min: 0.03586565913629336 iter_num: 3999
进度: 12/32  error_min: 8.979772590400614e-06 iter_num: 3513
进度: 13/32  error_min: 9.67012178487714e-06 iter_num: 731
进度: 14/32  error_min: 1.9607755619599487e-05 iter_num: 3999
进度: 15/32  error_min: 8.450028221917982e-06 iter_num: 773
进度: 16/32  error_min: 0.00678369144508828 iter_num: 3999
进度: 17/32  error_min: 5.758011676459951e-06 iter_num: 1195
进度: 18/32  error_m

In [27]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
# circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 8.548877371783448e-06 iter_num: 2115
进度: 2/32  error_min: 9.577031847118889e-06 iter_num: 2089
进度: 3/32  error_min: 9.82120537118547e-06 iter_num: 285
进度: 4/32  error_min: 0.009429408909622738 iter_num: 3999
进度: 5/32  error_min: 9.450679860578681e-06 iter_num: 1388
进度: 6/32  error_min: 8.68603677617319e-06 iter_num: 2174
进度: 7/32  error_min: 5.022051779768333e-06 iter_num: 485
进度: 8/32  error_min: 9.910529427337167e-06 iter_num: 3420
进度: 9/32  error_min: 0.00249722526260443 iter_num: 3999
进度: 10/32  error_min: 9.120734732248614e-06 iter_num: 484
进度: 11/32  error_min: 0.07550152165487056 iter_num: 3999
进度: 12/32  error_min: 2.4677548797757964e-06 iter_num: 2919
进度: 13/32  error_min: 9.201035537698132e-06 iter_num: 718
进度: 14/32  error_min: 6.855918965431584e-06 iter_num: 314
进度: 15/32  error_min: 9.214719092653922e-06 iter_num: 901
进度: 16/32  error_min: 6.733023525273829e-06 iter_num: 1691
进度: 17/32  error_min: 8.913809145250084e-06 iter_num: 1890
进度: 18/32  error_m

In [26]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
# circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 6.118761978979315e-06 iter_num: 335
进度: 2/32  error_min: 8.234375922122616e-06 iter_num: 1864
进度: 3/32  error_min: 8.720360216041279e-06 iter_num: 2529
进度: 4/32  error_min: 0.018545381783878434 iter_num: 3999
进度: 5/32  error_min: 6.967917017597358e-06 iter_num: 1738
进度: 6/32  error_min: 9.955042843623474e-06 iter_num: 2151
进度: 7/32  error_min: 8.864843651479326e-06 iter_num: 506
进度: 8/32  error_min: 3.7443956298899295e-06 iter_num: 2947
进度: 9/32  error_min: 8.424956932873151e-06 iter_num: 1161
进度: 10/32  error_min: 1.5599887684958347e-06 iter_num: 521
进度: 11/32  error_min: 0.007955083828605436 iter_num: 3999
进度: 12/32  error_min: 0.001938082983555911 iter_num: 3999
进度: 13/32  error_min: 8.55379760456021e-06 iter_num: 313
进度: 14/32  error_min: 9.581085096743713e-06 iter_num: 1420
进度: 15/32  error_min: 9.294505045498447e-06 iter_num: 332
进度: 16/32  error_min: 6.180323825044631e-06 iter_num: 1020
进度: 17/32  error_min: 6.5557211746947175e-06 iter_num: 539
进度: 18/32  er

In [25]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
# circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 2.906626915155641e-06 iter_num: 2412
进度: 2/32  error_min: 7.268805465754369e-06 iter_num: 1378
进度: 3/32  error_min: 9.609328356141589e-06 iter_num: 729
进度: 4/32  error_min: 4.347346867272783e-06 iter_num: 3617
进度: 5/32  error_min: 5.372868715602941e-06 iter_num: 535
进度: 6/32  error_min: 9.475299826311456e-06 iter_num: 1892
进度: 7/32  error_min: 9.635064586954734e-06 iter_num: 302
进度: 8/32  error_min: 8.738703307753504e-06 iter_num: 276
进度: 9/32  error_min: 9.23702258637249e-06 iter_num: 2573
进度: 10/32  error_min: 9.096738704972296e-06 iter_num: 320
进度: 11/32  error_min: 0.032770047316698525 iter_num: 3999
进度: 12/32  error_min: 4.8776644105963385e-06 iter_num: 1062
进度: 13/32  error_min: 9.747907629198416e-06 iter_num: 427
进度: 14/32  error_min: 8.36114599322979e-06 iter_num: 953
进度: 15/32  error_min: 9.748525600206115e-06 iter_num: 369
进度: 16/32  error_min: 7.894019974807875e-06 iter_num: 1052
进度: 17/32  error_min: 7.653891384795664e-06 iter_num: 466
进度: 18/32  error_

In [24]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
# circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 8.624528129330677e-06 iter_num: 549
进度: 2/32  error_min: 6.1987380224337585e-06 iter_num: 1260
进度: 3/32  error_min: 9.146348280530958e-06 iter_num: 1298
进度: 4/32  error_min: 1.6565955861524984e-06 iter_num: 594
进度: 5/32  error_min: 9.458303201537177e-06 iter_num: 466
进度: 6/32  error_min: 6.6519643422413566e-06 iter_num: 799
进度: 7/32  error_min: 8.748668846814134e-06 iter_num: 879
进度: 8/32  error_min: 7.647590315906783e-06 iter_num: 766
进度: 9/32  error_min: 7.778569532157498e-06 iter_num: 172
进度: 10/32  error_min: 8.417947746419685e-06 iter_num: 1073
进度: 11/32  error_min: 0.0005671191161752409 iter_num: 3999
进度: 12/32  error_min: 7.812141058827216e-06 iter_num: 1128
进度: 13/32  error_min: 9.679926145755857e-06 iter_num: 741
进度: 14/32  error_min: 3.1021892755722646e-06 iter_num: 2196
进度: 15/32  error_min: 9.439310558412295e-06 iter_num: 326
进度: 16/32  error_min: 8.178728588381645e-06 iter_num: 1100
进度: 17/32  error_min: 8.874706581729619e-06 iter_num: 310
进度: 18/32  e

In [23]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

# circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 5.156942767814421e-06 iter_num: 283
进度: 2/32  error_min: 9.591723020063725e-06 iter_num: 752
进度: 3/32  error_min: 6.1277449416863305e-06 iter_num: 359
进度: 4/32  error_min: 4.706906689855295e-06 iter_num: 1912
进度: 5/32  error_min: 7.984503160529677e-06 iter_num: 944
进度: 6/32  error_min: 6.658619882138339e-06 iter_num: 3376
进度: 7/32  error_min: 9.835333197316665e-06 iter_num: 712
进度: 8/32  error_min: 9.365616177259284e-06 iter_num: 241
进度: 9/32  error_min: 8.530737460321625e-06 iter_num: 537
进度: 10/32  error_min: 7.393486258111537e-06 iter_num: 758
进度: 11/32  error_min: 9.260007145184268e-06 iter_num: 733
进度: 12/32  error_min: 5.001205450438917e-06 iter_num: 2737
进度: 13/32  error_min: 8.296906001348248e-06 iter_num: 447
进度: 14/32  error_min: 5.583355962568426e-06 iter_num: 322
进度: 15/32  error_min: 4.821026057255651e-06 iter_num: 418
进度: 16/32  error_min: 2.120018315454608e-06 iter_num: 1065
进度: 17/32  error_min: 7.165178131152139e-06 iter_num: 262
进度: 18/32  error_m

In [22]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 2.8627235193479095e-07 iter_num: 948
进度: 2/32  error_min: 4.232382480595653e-06 iter_num: 615
进度: 3/32  error_min: 9.014042473665107e-06 iter_num: 130
进度: 4/32  error_min: 5.9140518340328185e-06 iter_num: 777
进度: 5/32  error_min: 5.859669840013737e-06 iter_num: 676
进度: 6/32  error_min: 0.0003023491705842485 iter_num: 3999
进度: 7/32  error_min: 7.260193236380275e-06 iter_num: 572
进度: 8/32  error_min: 7.320080003370144e-06 iter_num: 1911
进度: 9/32  error_min: 6.3506511415178934e-06 iter_num: 257
进度: 10/32  error_min: 6.61675887925206e-06 iter_num: 492
进度: 11/32  error_min: 0.00028807438262234975 iter_num: 3999
进度: 12/32  error_min: 7.514627659244155e-06 iter_num: 727
进度: 13/32  error_min: 2.0465427778004752e-06 iter_num: 934
进度: 14/32  error_min: 5.5684219645302235e-06 iter_num: 234
进度: 15/32  error_min: 8.22938415534491e-06 iter_num: 307
进度: 16/32  error_min: 7.0235148740271924e-06 iter_num: 942
进度: 17/32  error_min: 6.7772918365127666e-06 iter_num: 669
进度: 18/32  err

In [21]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 8.84668157219437e-06 iter_num: 400
进度: 2/32  error_min: 8.53526304822605e-06 iter_num: 743
进度: 3/32  error_min: 6.6813306326407584e-06 iter_num: 1066
进度: 4/32  error_min: 7.346130531327155e-06 iter_num: 551
进度: 5/32  error_min: 9.388669918286574e-06 iter_num: 474
进度: 6/32  error_min: 3.710374978482811e-06 iter_num: 1095
进度: 7/32  error_min: 3.617098549568709e-06 iter_num: 446
进度: 8/32  error_min: 7.708152753416186e-06 iter_num: 425
进度: 9/32  error_min: 1.3573529592525446e-06 iter_num: 546
进度: 10/32  error_min: 6.948863480848999e-06 iter_num: 1463
进度: 11/32  error_min: 4.811637173918193e-06 iter_num: 652
进度: 12/32  error_min: 5.800298286118277e-06 iter_num: 412
进度: 13/32  error_min: 9.423738944192017e-06 iter_num: 938
进度: 14/32  error_min: 6.572347996280925e-06 iter_num: 538
进度: 15/32  error_min: 9.136268624865274e-06 iter_num: 182
进度: 16/32  error_min: 9.109511907912093e-06 iter_num: 443
进度: 17/32  error_min: 3.791093626359654e-06 iter_num: 464
进度: 18/32  error_min

In [8]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
circ += gate_0('11').on(0)
circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 8.018508700136096e-06 iter_num: 369
进度: 2/32  error_min: 9.569743167125999e-06 iter_num: 293
进度: 3/32  error_min: 9.625729032625152e-06 iter_num: 500
进度: 4/32  error_min: 9.140834981380408e-06 iter_num: 1125
进度: 5/32  error_min: 8.023507596544022e-06 iter_num: 495
进度: 6/32  error_min: 8.843959725890471e-06 iter_num: 1013
进度: 7/32  error_min: 9.55982539185829e-06 iter_num: 642
进度: 8/32  error_min: 8.388071472631431e-06 iter_num: 956
进度: 9/32  error_min: 1.76262908790914e-06 iter_num: 1356
进度: 10/32  error_min: 4.943593907680288e-06 iter_num: 370
进度: 11/32  error_min: 9.305474427123706e-06 iter_num: 439
进度: 12/32  error_min: 7.031385551914404e-06 iter_num: 668
进度: 13/32  error_min: 9.386661403709695e-06 iter_num: 427
进度: 14/32  error_min: 8.75947444112235e-06 iter_num: 306
进度: 15/32  error_min: 8.782660634087947e-06 iter_num: 200
进度: 16/32  error_min: 3.01296733129508e-06 iter_num: 1809
进度: 17/32  error_min: 6.078597599934454e-06 iter_num: 1315
进度: 18/32  error_min: 

In [28]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
circ += gate_0('11').on(0)
circ += gate_0('12').on(0)
circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 2.9176593930868577e-06 iter_num: 492
进度: 2/32  error_min: 9.707463507591108e-06 iter_num: 483
进度: 3/32  error_min: 5.363244803491263e-06 iter_num: 329
进度: 4/32  error_min: 7.100093438849875e-06 iter_num: 683
进度: 5/32  error_min: 7.513125426128475e-06 iter_num: 611
进度: 6/32  error_min: 4.730430267718688e-06 iter_num: 1899
进度: 7/32  error_min: 5.264405637417369e-06 iter_num: 253
进度: 8/32  error_min: 7.982269009110254e-06 iter_num: 906
进度: 9/32  error_min: 8.675682383163519e-06 iter_num: 776
进度: 10/32  error_min: 7.610013475600752e-06 iter_num: 812
进度: 11/32  error_min: 8.372992603478124e-06 iter_num: 1511
进度: 12/32  error_min: 7.4001078621543925e-06 iter_num: 522
进度: 13/32  error_min: 3.530089060066466e-06 iter_num: 1650
进度: 14/32  error_min: 9.194134762302575e-06 iter_num: 727
进度: 15/32  error_min: 9.594590660078062e-06 iter_num: 290
进度: 16/32  error_min: 4.794364383275962e-07 iter_num: 847
进度: 17/32  error_min: 8.44556339429925e-06 iter_num: 623
进度: 18/32  error_mi

In [16]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
circ += gate_0('11').on(0)
circ += gate_0('12').on(0)
circ += gate_0('13').on(0)
circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 9.6121552672912e-06 iter_num: 641
进度: 2/32  error_min: 5.035774005013316e-06 iter_num: 560
进度: 3/32  error_min: 9.160490345383465e-06 iter_num: 933
进度: 4/32  error_min: 9.085394180208262e-06 iter_num: 737
进度: 5/32  error_min: 7.443008824603581e-06 iter_num: 671
进度: 6/32  error_min: 5.793386794405997e-06 iter_num: 450
进度: 7/32  error_min: 9.061412234334654e-06 iter_num: 1016
进度: 8/32  error_min: 7.558411317720726e-06 iter_num: 1030
进度: 9/32  error_min: 7.159125578692915e-06 iter_num: 626
进度: 10/32  error_min: 2.178361631743897e-06 iter_num: 1784
进度: 11/32  error_min: 9.457726972472713e-06 iter_num: 472
进度: 12/32  error_min: 4.328448272805652e-06 iter_num: 721
进度: 13/32  error_min: 7.762338349093234e-06 iter_num: 615
进度: 14/32  error_min: 6.763171499946097e-06 iter_num: 598
进度: 15/32  error_min: 2.267615826112568e-06 iter_num: 607
进度: 16/32  error_min: 9.644420654653096e-06 iter_num: 695
进度: 17/32  error_min: 9.079189811633803e-06 iter_num: 140
进度: 18/32  error_min: 

In [15]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
circ += gate_0('11').on(0)
circ += gate_0('12').on(0)
circ += gate_0('13').on(0)
circ += gate_0('14').on(0)
circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.05
for i in range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 1/32  error_min: 3.4253672188677697e-06 iter_num: 694
进度: 2/32  error_min: 4.224228158045129e-06 iter_num: 455
进度: 3/32  error_min: 7.233869388723058e-06 iter_num: 264
进度: 4/32  error_min: 2.698849282833926e-06 iter_num: 904
进度: 5/32  error_min: 5.809239904452923e-06 iter_num: 399
进度: 6/32  error_min: 7.017408601206476e-06 iter_num: 395
进度: 7/32  error_min: 4.255298976962862e-06 iter_num: 305
进度: 8/32  error_min: 7.315566393994999e-06 iter_num: 713
进度: 9/32  error_min: 6.157588105293321e-06 iter_num: 277
进度: 10/32  error_min: 2.958569227939911e-06 iter_num: 1035
进度: 11/32  error_min: 9.985108710863777e-06 iter_num: 465
进度: 12/32  error_min: 8.513927841957702e-06 iter_num: 320
进度: 13/32  error_min: 8.585133563054903e-06 iter_num: 789
进度: 14/32  error_min: 4.4132685222963985e-06 iter_num: 262
进度: 15/32  error_min: 6.71784613215376e-06 iter_num: 454
进度: 16/32  error_min: 7.369638270859902e-06 iter_num: 373
进度: 17/32  error_min: 4.720847349370949e-06 iter_num: 1115
进度: 18/32  error_min